In [1]:
import torch
from torch.utils.data import DataLoader
from utils import make_regression_data, mse, log_epoch, RegressionDataset

class LinearRegressionManual:
    def __init__(self, in_features):
        self.w = torch.randn(in_features, 1, dtype=torch.float32, requires_grad=False)
        self.b = torch.zeros(1, dtype=torch.float32, requires_grad=False)

    def __call__(self, X):
        return X @ self.w + self.b

    def parameters(self):
        return [self.w, self.b]

    def zero_grad(self):
        self.dw = torch.zeros_like(self.w)
        self.db = torch.zeros_like(self.b)
    

    def l1_regularization(self):
        """L1-регуляризация (Lasso): возвращает сумму |w|"""
        return torch.sum(torch.abs(self.w))

    def l2_regularization(self):
        """L2-регуляризация (Ridge): возвращает сумму w^2"""
        return torch.sum(self.w ** 2)

    def backward(self, X, y, y_pred):
        n = X.shape[0]
        error = y_pred - y
        self.dw = (X.T @ error) / n
        self.db = error.mean(0)

        if self.l1_lambda > 0:
            self.dw += self.l1_lambda * torch.sign(self.w)  # Производная |w| = sign(w)
        if self.l2_lambda > 0:
            self.dw += 2 * self.l2_lambda * self.w         # Производная w^2 = 2w
        
        

    def step(self, lr):
        self.w -= lr * self.dw
        self.b -= lr * self.db

    def set_l1_lambda(self, lambda_):
        """Установка коэффициента L1-регуляризации"""
        self.l1_lambda = lambda_

    def set_l2_lambda(self, lambda_):
        """Установка коэффициента L2-регуляризации"""
        self.l2_lambda = lambda_

    def save(self, path):
        torch.save({'w': self.w, 'b': self.b}, path)

    def load(self, path):
        state = torch.load(path)
        self.w = state['w']
        self.b = state['b']


In [2]:
if __name__ == '__main__':
    # Генерируем данные
    X, y = make_regression_data(n=200)
    
    # Создаём датасет и даталоадер
    dataset = RegressionDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    print(f'Размер датасета: {len(dataset)}')
    print(f'Количество батчей: {len(dataloader)}')
    print(f'Пример данных: {dataset[0]}')
    
    # Обучаем модель
    model = LinearRegressionManual(in_features=1)
    model.set_l1_lambda(0.01)  # Включить L1 с lambda=0.01
    model.set_l2_lambda(0.001) # Включить L2 с lambda=0.001
    lr = 0.1
    epochs = 100
    best_loss = float("inf")
    waiting_Epoch = 10
    no_improve = 0

    

    for epoch in range(1, epochs + 1):
        total_loss = 0
        
        for i, (batch_X, batch_y) in enumerate(dataloader):
            y_pred = model(batch_X)
            loss = mse(y_pred, batch_y)
            total_loss += loss + model.l1_lambda * model.l1_regularization()
            total_loss += model.l2_lambda * model.l2_regularization()
            
            model.zero_grad()
            model.backward(batch_X, batch_y, y_pred)
            model.step(lr)
            
        
        avg_loss = total_loss / (i + 1)
        
        if epoch % 10 == 0:
            log_epoch(epoch, avg_loss)
        
        if (best_loss-avg_loss)>0.001:
            best_loss = avg_loss
            no_improve = 0

        else:
            no_improve+=1
            if waiting_Epoch< no_improve:

                print(epoch,best_loss)
                break
        

        
        
    
    model.save('linreg_manual.pth')

Размер датасета: 200
Количество батчей: 7
Пример данных: (tensor([0.4992]), tensor([-0.0954]))
Epoch 10: loss=0.2361
Epoch 20: loss=0.1245
Epoch 30: loss=0.0738
Epoch 40: loss=0.0540
Epoch 50: loss=0.0445
Epoch 60: loss=0.0392
Epoch 70: loss=0.0369
Epoch 80: loss=0.0363
83 tensor(0.0361)
